In [10]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import numpy as np

In [11]:
fem = pd.read_csv(r'traning_data')
aim = pd.read_csv(r'tranig_data')
privbayes = pd.read_csv(r'tranig_data')
mst = pd.read_csv(r'tranig_data')
mwem = pd.read_csv(r'tranig_data')
dpgan = pd.read_csv(r'tranig_data')
dpctgan = pd.read_csv(r'tranig_data')
pategan = pd.read_csv(r'tranig_data')
patectgan = pd.read_csv(r'tranig_data')

In [12]:
def train_and_evaluate(dataset, test_data, target_column, n_iterations=10):
    auroc_scores = []
    
    # Split the dataset into features and target
    X = dataset.drop(columns=[target_column])
    y = dataset[target_column]
    
    # Check if the target variable has both classes (0 and 1)
    if len(y.unique()) < 2:
        print(f"Skipping dataset - Target variable has only one class: {y.unique()}")
        return None
    
    # Split the test dataset into features and target
    X_test = test_data.drop(columns=[target_column])
    y_test = test_data[target_column]
    
    for i in range(n_iterations):
        # Split training data for training and validation
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=i)
        
        # Create a fresh XGBoost model
        model = xgb.XGBClassifier(eval_metric='logloss', random_state=i)
        
        # Train the model
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        
        # Predict probabilities for the test data
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        
        # Calculate AUROC score
        auroc = roc_auc_score(y_test, y_pred_proba)
        auroc_scores.append(auroc)
    
    # Return the average AUROC score across iterations
    return np.mean(auroc_scores)

In [13]:
target_column = 'label'

In [14]:
pp_csv = pd.read_csv('test_file')

In [15]:
# Define the dataset names corresponding to your datasets
dataset_names = ["FEM","AIM","PrivBayes", "MST", "MWEM","DPGAN","DPCTGAN", "DPCTGAN","PATECTGAN"]
datasets = [fem,aim, privbayes, mst, mwem, dpgan, dpctgan,pategan, patectgan]

# Loop through datasets with their names
for dataset_name, dataset in zip(dataset_names, datasets):
    print(f"Processing Dataset {dataset_name}...")
    auroc = train_and_evaluate(dataset, pp_csv, target_column)
    if auroc is not None:
        print(f"{dataset_name}: Average AUROC Score: {auroc:.4f}")
    else:
        print(f"{dataset_name}: Skipped due to insufficient target class diversity.")


Processing Dataset FEM...
FEM: Average AUROC Score: 0.5859
Processing Dataset AIM...
AIM: Average AUROC Score: 0.6911
Processing Dataset PrivBayes...
PrivBayes: Average AUROC Score: 0.6302
Processing Dataset MST...
MST: Average AUROC Score: 0.5349
Processing Dataset MWEM...
MWEM: Average AUROC Score: 0.6551
Processing Dataset DPGAN...
Skipping dataset - Target variable has only one class: [1]
DPGAN: Skipped due to insufficient target class diversity.
Processing Dataset DPCTGAN...
DPCTGAN: Average AUROC Score: 0.5959
Processing Dataset DPCTGAN...
DPCTGAN: Average AUROC Score: 0.4026
Processing Dataset PATECTGAN...
PATECTGAN: Average AUROC Score: 0.5374
